<a href="https://colab.research.google.com/github/kunalmaurya-17-24/2d-Solar-System/blob/master/langgraph_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.


In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Install LaTeX compiler and packages
!apt-get update
!apt-get install -y texlive-latex-recommended texlive-latex-extra texlive-xetex

# Install Python libraries for LaTeX parsing/editing
!pip install TexSoup pylatexenc

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,822 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 http://

In [5]:
!pip install tex2py

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.7/284.7 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for coverage: filename=coverage-3.7.1-py3-none-any.whl size=109786 sha256=89f5dce9b5724c126a623dfbcbf81b9bf785123953301c8a96b6fe8804e1e6d1
  Stored in directory: /root/.cache/pip/wheels/6d/e8/a0/03d0d1daeffded38c51df6a26b01349830f48382aac5515248
  Created wheel for pptree: filename=pptree-2.0-py3-none-any.whl size=2323 sha256=48852cb0978f932549bd648063df8c12c91c4c1be7ac1b32da86cff67ece4885
  Stored in directory: /root/.cache/pip/wheels/8d/31/65/cdb399c1e0df53fb4022c80d732d66052e43aa4c83c078eedc
  Created wheel for TexSoup: filename=TexSoup-0.1.4-py3-none-any.whl size=14641 sha256=a721fb96d8698374fbb4ee4e3a1bcee99dbfe90605baa6e8fa86cb434ea75f35
  Stored in directory: /root/.cache/pip/wheels/64/14/90/22b9831ae9c785a0aa54bda49b860a9

In [8]:
from google.colab import files
uploaded = files.upload()  # Select your LaTeX .tex file

Saving your_resume.tex to your_resume (2).tex


In [10]:
from TexSoup import TexSoup

with open("/content/your_resume.tex") as f:
    latex_doc = f.read()

soup = TexSoup(latex_doc)
# Now you can inspect and edit sections, commands, environments, etc.


In [11]:
import fitz  # PyMuPDF

doc = fitz.open("/content/Kunal Maurya CV.pdf")

# Extract text
text = ""
for page in doc:
    text += page.get_text()

# Extract metadata
metadata = doc.metadata

print("---- Resume Text ----")
print(text)

print("\n---- PDF Metadata ----")
for key, value in metadata.items():
    print(f"{key}: {value}")


ModuleNotFoundError: No module named 'fitz'

In [12]:
import fitz  # For PDF text/metadata extraction
from TexSoup import TexSoup  # For LaTeX parsing/editing
import subprocess  # For compiling LaTeX to PDF

class ResumeTailorAgent:
    def __init__(self, resume_pdf_path=None, resume_tex_path=None, drive_path=None):
        self.resume_pdf_path = resume_pdf_path
        self.resume_tex_path = resume_tex_path
        self.drive_path = drive_path
        self.metadata = None
        self.resume_text = None
        self.latex_source = None

    def load_resume(self):
        if self.resume_pdf_path:
            doc = fitz.open(self.resume_pdf_path)
            self.metadata = doc.metadata
            self.resume_text = "".join([page.get_text() for page in doc])
        if self.resume_tex_path:
            with open(self.resume_tex_path) as f:
                self.latex_source = f.read()
        return self.resume_text or self.latex_source or ""

    def load_job_description(self, jd_text):
        self.job_description = jd_text

    def extract_keywords(self):
        # Replace with your NLP keyword extraction pipeline (using spacy/transformer/regex)
        self.jd_keywords = ["machine learning", "python", "nlp"]  # Example, plug in from NLP
        return self.jd_keywords

    def suggest_changes(self):
        # Compare resume (text or latex_source) to JD keywords
        missing = [kw for kw in self.jd_keywords if kw.lower() not in (self.resume_text or self.latex_source).lower()]
        # Generate suggested edits (this is where your analysis logic goes)
        suggestions = {
            "missing_keywords": missing,
            "sections_to_modify": ["Skills", "Projects"],
            "raw_resume_text": self.resume_text,
            "metadata": self.metadata
        }
        return suggestions

    def update_latex_resume(self):
        if not self.latex_source:
            raise Exception("No LaTeX source loaded!")
        soup = TexSoup(self.latex_source)
        # Example: Insert missing keywords into Skills section
        # Implementation depends on your resume template structure
        # ...
        self.updated_latex = str(soup)  # After modifications
        return self.updated_latex

    def compile_to_pdf(self, output_name="updated_resume.pdf"):
        # Write updated LaTeX to file
        with open("resume_updated.tex", "w") as f:
            f.write(self.updated_latex)
        # Compile via pdflatex
        subprocess.run(["pdflatex", "resume_updated.tex"])
        # Move to Google Drive (if needed)
        if self.drive_path:
            import shutil
            shutil.move("resume_updated.pdf", f"{self.drive_path}/resume_updated.pdf")
        return "resume_updated.pdf"

    def run(self, jd_text):
        self.load_resume()
        self.load_job_description(jd_text)
        self.extract_keywords()
        suggestions = self.suggest_changes()
        self.update_latex_resume()
        pdf_path = self.compile_to_pdf()
        return {
            "suggestions": suggestions,
            "pdf_path": pdf_path
        }

# Example usage:
# agent = ResumeTailorAgent(resume_pdf_path="/content/CV.pdf", resume_tex_path="/content/CV.tex", drive_path="/content/drive/My Drive")
# result = agent.run(jd_text="We need a Python developer skilled in NLP and machine learning")
# print(result["suggestions"])
# print("Saved to:", result["pdf_path"])


ModuleNotFoundError: No module named 'fitz'

In [13]:
# Install TexSoup for LaTeX parsing (if not already done)
!pip install TexSoup

import json
from TexSoup import TexSoup

class ResumeContextAgent:
    def __init__(self):
        self.structure = {}
        self.latex_source = ""
        self.mem_file = "resume_structure.json"

    def parse_latex_resume(self, tex_path):
        with open(tex_path, "r") as f:
            self.latex_source = f.read()
        soup = TexSoup(self.latex_source)
        # Parse high-level sections using common LaTeX commands
        sections = []
        for section in soup.find_all('section'):
            # Each section may have a name and content
            sections.append({'name': str(section.string).strip(), 'content': str(section.args if section.args else section.contents)})
        self.structure = {
            "sections": sections,
        }
        # Save structure persistently
        with open(self.mem_file, "w") as out:
            json.dump(self.structure, out)
        print("Initial resume structure saved (sections in correct order):")
        for s in self.structure["sections"]:
            print("-", s['name'])
        return self.structure

    def load_structure_from_memory(self):
        with open(self.mem_file, "r") as f:
            self.structure = json.load(f)
        return self.structure

    def get_section_names(self):
        if not self.structure:
            self.load_structure_from_memory()
        return [sec["name"] for sec in self.structure["sections"]]

# Usage example (first run):
resume_agent = ResumeContextAgent()
resume_agent.parse_latex_resume("your_resume.tex")

# In future runs:
stored_structure = resume_agent.load_structure_from_memory()
print("Stored section order:", resume_agent.get_section_names())


Initial resume structure saved (sections in correct order):
- Education
- Experience
- Projects
- Technical Skills
- Achievements \& Certifications
Stored section order: ['Education', 'Experience', 'Projects', 'Technical Skills', 'Achievements \\& Certifications']


In [14]:
# Install what you need:
!pip install keybert transformers sentence-transformers spacy

import spacy
from keybert import KeyBERT
from transformers import pipeline

# Load open-source spaCy model and keyBERT
nlp = spacy.load("en_core_web_sm")
kw_model = KeyBERT()

def extract_nouns_verbs(text):
    # Quickly pull out noun/verb phrases
    doc = nlp(text)
    return [chunk.text.lower() for chunk in doc.noun_chunks] + [token.lemma_ for token in doc if token.pos_ == "VERB"]

def extract_keywords(text):
    # Use keyBERT for more accurate keyword extraction
    return kw_model.extract_keywords(text, keyphrase_ngram_range=(1,2), stop_words='english', top_n=20)

def compare_resume_to_jd(resume_sections, jd_text):
    jd_keywords = set([kw[0] for kw in extract_keywords(jd_text)])
    match_report = {}
    for sec in resume_sections:
        section_text = sec["content"]
        section_keywords = set([kw[0] for kw in extract_keywords(section_text)])
        match_score = len(jd_keywords.intersection(section_keywords)) / max(1, len(jd_keywords))
        match_report[sec["name"]] = {"score": match_score, "missing": list(jd_keywords - section_keywords)}
    return match_report

# (Optional) Open-source LLM for rewriting (T5, TinyLlama, DeepSeek, etc.)
gen = pipeline("text2text-generation", model="google/flan-t5-base")

def rewrite_section(section_text, prompt):
    return gen(
        f"Rewrite the following as if it perfectly fits this job: {prompt}\nText: {section_text}",
        max_length=256
    )[0]["generated_text"]

# Usage:
initial_structure = resume_agent.load_structure_from_memory()
# # jd = """(paste your job description here)"""
# from google.colab import widgets
# jd = widgets.Textarea(value='', placeholder='Paste your job description here...')
# print("Paste your job description in the cell above and run the next step.")
jd = input("Paste your job description here (single line, or copy-paste the whole block, then press Enter):\n")



report = compare_resume_to_jd(initial_structure["sections"], jd)
print(report)

# For each weak/missing section:
# updated_content = rewrite_section(sec['content'], jd)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Paste your job description here (single line, or copy-paste the whole block, then press Enter):
DataAnnotation is committed to creating quality AI. Join our team to help train AI chatbots while gaining the flexibility of remote work and choosing your own schedule.  We are looking for a bilingual professional to join our team and teach AI chatbots. You will have conversations in both Hindi and English with chatbots in order to measure their progress, as well as write novel conversations in order to teach them what to say.  Benefits:  This is a full-time or part-time REMOTE position You’ll be able to choose which projects you want to work on You can work on your own schedule Projects are paid hourly, starting at $22+ USD per hour, with bonuses for high-quality and high-volume work Responsibilities (both in Hindi and English):  Come up with diverse conversations over a range of topics Write high-quality answers when given specific prompts Compare the performance of different AI models Res

In [15]:
new_sections = []
for sec in initial_structure["sections"]:
    missing = report[sec["name"]]["missing"]
    # Use missing keywords as injection prompt
    prompt = f"Include these job-specific keywords, skills, or responsibilities in the section if relevant: {', '.join(missing)}."
    # You could supply the original content or a template placeholder
    original = sec["content"] if sec["content"] != f"{{{sec['name']}}}" else ""
    # Rewrite section to maximize JD match
    rewritten = rewrite_section(original, jd + " " + prompt)
    new_sections.append({'name': sec["name"], 'content': rewritten})


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [16]:
def build_latex_resume(sections):
    template_top = r"""
\documentclass[10pt]{article}
\begin{document}
"""
    template_bottom = r"\end{document}"
    body = ""
    for sec in sections:
        body += f"\\section*{{{sec['name']}}}\n{sec['content']}\n"
    return template_top + body + template_bottom

latex_source = build_latex_resume(new_sections)
with open("tailored_resume.tex", "w") as f:
    f.write(latex_source)


In [17]:
import subprocess
subprocess.run(["pdflatex", "tailored_resume.tex"])
# Optionally move to Google Drive, renaming as needed
import shutil
company = "DataAnnotation"
position = "AI_Chatbot_Trainer"
output_name = f"{company}_{position}.pdf"
shutil.move("tailored_resume.pdf", f"/content/drive/My Drive/{output_name}")


FileNotFoundError: [Errno 2] No such file or directory: 'tailored_resume.pdf'